In [6]:
import torch
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

from torch import nn
from d2l import torch as d2l

batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)   # 这块自己写真复杂呀

In [7]:
for x, y in train_iter:
    print(x, y)
    exit(-1)

tensor([[ 3,  9,  2,  ...,  2,  1,  3],
        [15,  4, 14,  ...,  4,  6, 11],
        [17,  5,  3,  ...,  1,  4,  6],
        ...,
        [ 3,  1,  4,  ..., 12,  2, 13],
        [ 6,  3, 15,  ...,  5,  1, 13],
        [ 9,  2,  1,  ...,  4,  5, 10]]) tensor([[ 9,  2,  1,  ...,  1,  3,  5],
        [ 4, 14, 18,  ...,  6, 11, 20],
        [ 5,  3,  9,  ...,  4,  6, 11],
        ...,
        [ 1,  4,  1,  ...,  2, 13,  1],
        [ 3, 15, 10,  ...,  1, 13, 14],
        [ 2,  1, 13,  ...,  5, 10,  1]])
tensor([[ 5, 13,  2,  ..., 15,  7,  6],
        [20,  4,  8,  ...,  1, 21,  2],
        [11,  1, 12,  ...,  7, 22,  2],
        ...,
        [ 1, 21, 14,  ..., 10,  2, 16],
        [14,  8,  3,  ..., 12,  7, 17],
        [ 1,  4,  6,  ...,  2, 15,  3]]) tensor([[13,  2,  1,  ...,  7,  6, 22],
        [ 4,  8,  8,  ..., 21,  2,  5],
        [ 1, 12,  4,  ..., 22,  2, 10],
        ...,
        [21, 14,  3,  ...,  2, 16,  4],
        [ 8,  3,  1,  ...,  7, 17,  8],
        [ 4,  6, 11,  ...

In [8]:
def get_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size
    
    def normal(shape):
        return torch.randn(size=shape, device=device) * .01     # 均值0、方差.01

    def three():
        return (normal((num_inputs, num_hiddens)),
                normal((num_hiddens, num_hiddens)),
                torch.zeros(num_hiddens, device=device))
    
    W_xz, W_hz, b_z = three()   # 更新门参数
    W_xr, W_hr, b_r = three()   # 重置门参数
    W_xh, W_hh, b_h = three()   # 候选隐状态参数
    # 输出层参数
    W_hq = normal((num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs, device=device)

    params = [W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

In [9]:
def init_gru_state(batch_size, num_hiddens, device):
    return (torch.zeros(size=(batch_size, num_hiddens), device=device), )

In [10]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = torch.sigmoid((X @ W_xz) + (H @ W_hz) + b_z)
        R = torch.sigmoid((X @ W_xr) + (H @ W_hr) + b_r)
        H_tilda = torch.tanh((X @ W_xh) + ((R * H) @ W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = H @ W_hq + b_q
        outputs.append(Y)
    return torch.cat(outputs, dim=0), (H, )

In [11]:
vocab_size, num_hiddens, device = len(vocab), 256, d2l.try_gpu()
num_epochs, lr = 500, 1
model = d2l.RNNModelScratch(vocab_size, num_hiddens, device, get_params, init_gru_state, gru)
# d2l.train_ch8(model, train_iter, vocab, lr, num_epochs, device)

In [12]:
vocab_size

28